In [85]:
!pip install fastf1 pandas numpy matplotlib seaborn scikit-learn plotly

In [87]:
# EXPLANATION:
# This cell installs necessary packages and imports them
# - fastf1: Official F1 API for real-time and historical data
# - pandas: Data manipulation and analysis
# - numpy: Numerical computing
# - matplotlib/seaborn: Data visualization
# - scikit-learn: Machine learning algorithms



import fastf1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Added imports for our machine learning workflow
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

fastf1.Cache.enable_cache('f1_cache') # This caches data locally to avoid repeated downloads

# Let's also add a function to safely copy and show dataset changes
def show_dataframe_info(df, title="Dataset Info"):
    """Helper function to display dataset information"""
    print(f"\n=== {title} ===")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print(f"Memory usage: {df.memory_usage().sum() / 1024**2:.2f} MB")
    return df

def safe_copy_and_modify(original_df, title="Dataset Copy"):
    """Create a safe copy of dataframe and show info"""
    df_copy = original_df.copy()
    print(f"\n🔄 Created copy: {title}")
    show_dataframe_info(df_copy, f"Copy - {title}")
    return df_copy


In [83]:
# EXPLANATION:
# This function converts various time formats (lap times, qualifying times) 
# into seconds for easier numerical analysis
# F1 timing data comes in different formats, this standardizes them

def to_seconds(x):
    """Convert laptime / timedelta / string to seconds."""
    try:
        return pd.to_timedelta(x).total_seconds()
    except Exception:
        try:
            return float(x)
        except Exception:
            return np.nan

# Let's add some more utility functions for data exploration
def explore_column(df, column_name):
    """Explore a specific column in detail"""
    if column_name not in df.columns:
        print(f"❌ Column '{column_name}' not found!")
        return
    
    print(f"\n🔍 Exploring column: '{column_name}'")
    print(f"Data type: {df[column_name].dtype}")
    print(f"Non-null count: {df[column_name].count()}/{len(df)}")
    print(f"Unique values: {df[column_name].nunique()}")
    
    if df[column_name].dtype in ['object']:
        print(f"Sample values: {df[column_name].unique()[:10]}")
    else:
        print(f"Min: {df[column_name].min()}, Max: {df[column_name].max()}")
        print(f"Mean: {df[column_name].mean():.2f}")

def compare_datasets(original, modified, title="Comparison"):
    """Compare two datasets and show differences"""
    print(f"\n📊 {title}")
    print(f"Original shape: {original.shape}")
    print(f"Modified shape: {modified.shape}")
    print(f"Columns removed: {set(original.columns) - set(modified.columns)}")
    print(f"Columns added: {set(modified.columns) - set(original.columns)}")


In [45]:
# ===============================================================
# 3. Load race + qualifying session
# ===============================================================

def load_race_and_qual(year, gp_name):
    # Load race session
    race = fastf1.get_session(year, gp_name, 'R')
    race.load()

    rr = race.results.copy().reset_index(drop=True).rename(columns={
        'TeamName': 'Team',
        'GridPosition': 'Grid',
        'ClassifiedPosition': 'Position',
        'Number': 'DriverNumber'
    })

    # Try to extract race date safely
    race_date = pd.NaT
    for cand in (
        getattr(race, 'date', None),
        getattr(race, 'start_time', None),
        getattr(getattr(race, 'event', {}), 'EventDate', None),
        isinstance(getattr(race, 'event', None), dict) and race.event.get('EventDate')
    ):
        if cand is not None:
            race_date = pd.to_datetime(cand)
            break

    meta = {
        'RaceDate': race_date,
        'Event': getattr(getattr(race, 'event', {}), 'EventName', None) or gp_name,
        'Season': year
    }

    # Try qualifying
    qr = None
    try:
        qual = fastf1.get_session(year, gp_name, 'Q')
        qual.load()
        qr = qual.results.copy().reset_index(drop=True).rename(columns={
            'TeamName': 'Team',
            'ClassifiedPosition': 'QualPos',
            'GridPosition': 'QualGrid'
        })
        qr['QualTime_sec'] = qr['Q3'].apply(to_seconds)
    except Exception:
        # For sprint weekends, try SQ
        try:
            qual = fastf1.get_session(year, gp_name, 'SQ')
            qual.load()
            qr = qual.results.copy().reset_index(drop=True).rename(columns={
                'TeamName': 'Team',
                'ClassifiedPosition': 'QualPos',
                'GridPosition': 'QualGrid'
            })
            qr['QualTime_sec'] = qr['Q3'].apply(to_seconds)
        except Exception:
            qr = None

    return rr, qr, meta

In [65]:
#4
def assemble_prerace_features(rr, qr, meta, prior_results):
    df = rr.copy()
    df['RaceDate'] = meta['RaceDate']
    df['Event'] = meta['Event']
    df['Season'] = meta['Season']

    # Pick identifier (Driver Abbreviation or Number)
    if 'Abbreviation' in df.columns:
        driver_id_col = 'Abbreviation'
    elif 'DriverNumber' in df.columns:
        driver_id_col = 'DriverNumber'
    else:
        raise KeyError("No suitable driver identifier found (expected Abbreviation or DriverNumber).")

    # --- Clean Position column ---
    def clean_position(pos):
        try:
            return int(pos)
        except Exception:
            return 99   # e.g. DNF, NC, DSQ -> treat as poor finish
    df['Position'] = df['Position'].apply(clean_position)

    if not prior_results.empty and 'Position' in prior_results.columns:
        prior_results = prior_results.copy()
        prior_results['Position'] = prior_results['Position'].apply(clean_position)

    # --- Attach Qualifying grid ---
    if qr is not None and driver_id_col in qr.columns:
        qual_map = qr.set_index(driver_id_col)['QualGrid'].to_dict()
        df['QualGrid'] = df[driver_id_col].map(qual_map)
    else:
        df['QualGrid'] = df['Grid']

    # --- Constructor form (points up to now) ---
    team_points = (
        prior_results.groupby('Team')['Points'].sum()
        if not prior_results.empty else pd.Series(dtype=float)
    )
    df['ConstructorPointsSoFar'] = df['Team'].map(team_points).fillna(0)

    # --- Driver form (avg last 3 results) ---
    if not prior_results.empty and driver_id_col in prior_results.columns:
        driver_form = (
            prior_results.groupby(driver_id_col)['Position']
            .apply(lambda x: x.tail(3).mean())
        )
        df['DriverForm'] = df[driver_id_col].map(driver_form).fillna(10)
    else:
        df['DriverForm'] = 10

    # --- Ensure scalars are numeric ---
    df['Grid'] = pd.to_numeric(df['Grid'], errors='coerce')
    df['QualGrid'] = pd.to_numeric(df['QualGrid'], errors='coerce')
    df['ConstructorPointsSoFar'] = pd.to_numeric(df['ConstructorPointsSoFar'], errors='coerce')
    df['DriverForm'] = pd.to_numeric(df['DriverForm'], errors='coerce')

    return df

In [79]:
#5
def build_prerace_dataset(event_list):
    """
    Build pre-race dataset across multiple events.

    Parameters
    ----------
    event_list : list of tuples
        Example: [(2023, "Bahrain Grand Prix"), (2023, "Saudi Arabian Grand Prix"), ...]

    Returns
    -------
    pandas.DataFrame
        Features DataFrame with cleaned, standardized columns.
    """
    all_features = []
    prior_results = pd.DataFrame()

    for season, event in event_list:
        try:
            # Load event
            race = fastf1.get_event(season, event)
            race.load()

            # Race results
            rr = race.results.copy().reset_index(drop=True)

            # Qualifying results
            try:
                qual = race.get_qualifying()
                qr = qual[['Abbreviation', 'Grid']].copy()
                qr.rename(columns={'Grid': 'QualGrid'}, inplace=True)
            except Exception:
                qr = None

            # Metadata
            meta = {
                'Season': season,
                'Event': event,
                'RaceDate': race.date,
            }

            # Build features for this race
            features = assemble_prerace_features(rr, qr, meta, prior_results)

            all_features.append(features)

            # Update prior_results with cleaned data
            prior_results = pd.concat([prior_results, features], ignore_index=True)

        except Exception as e:
            print(f"Skipping {event} {season} due to error: {e}")

    if not all_features:
        raise ValueError("No features could be built. Check event list or data availability.")

    return pd.concat(all_features, ignore_index=True)

In [77]:
# ===============================================================
# 6. Example: build dataset for a few races
# ===============================================================

event_list = [
    (2023, "Bahrain Grand Prix"),
    (2023, "Saudi Arabian Grand Prix"),
    (2023, "Australian Grand Prix"),
    (2023, "Azerbaijan Grand Prix"),
    (2023, "Miami Grand Prix"),
]

features_df = build_prerace_dataset(event_list)
features_df.head()

Skipping Bahrain Grand Prix 2023 due to error: 'Event' object has no attribute 'load'
Skipping Saudi Arabian Grand Prix 2023 due to error: 'Event' object has no attribute 'load'
Skipping Australian Grand Prix 2023 due to error: 'Event' object has no attribute 'load'
Skipping Azerbaijan Grand Prix 2023 due to error: 'Event' object has no attribute 'load'
Skipping Miami Grand Prix 2023 due to error: 'Event' object has no attribute 'load'


ValueError: No features could be built. Check event list or data availability.